# Protein Veri Seti

## 1) Verinin Keşfi (EDA)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
protein = pd.read_csv("../input/protein-data-set/pdb_data_no_dups.csv")

* **Veri seti Proteinlerin yapısal dizinleri ile alakalidir.**

* **Veriler Protein Data Bank alinmistir.**

* **Yapisal biyologlar atomlarin molekuldeki yerlerini belirlemek icin "X-ray crystallography, NMR spectroscopy, and cryo-electron microscopy" bu uc yontemi kullanmistir ki bu yontemler bizim  degiskenlerimizin icerisinde barinmaktadir.**

* **structureId:** Yapisal ID.
* **classification:** Belirtilen kumeler, siniflar .
* **experimentalTechnique:** Arastirmada kullanilan teknikler, yontemler.
* **macromolecule Type:** Makromolekül Tipi. 
* **residueCount:** Tortu Sayisi.
* **resolution:** Bir bileşiğin, kendisini oluşturan unsurlara ayrılması; erime; çözünme; ayrışma.
* **structureMolecularWeight:** Yapinin molekul agirligidir.
* **crystallizationMethod:** Kristallestirme icin kullanilan metod. 
* **crystallizationTempK:** Kristallestirme sicakligi Kelvin cinsinden.
* **densityMatthews:** Kristale ait yogunluk.
* **densityPercentSol:** Yogunluk yuzdesi.
* **pdbxDetails:** pH degerine ait detaylar.
* **phValue:** pH degeri 
* **publicationYear:** Verinin yayinlanma yili.

In [ ]:
df = protein.copy()
df.head()

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

### 1.1) Kategorik Değişkenler

In [ ]:
dfkat = df.select_dtypes(include = ["object"])
dfkat

In [ ]:
dfkat.columns

In [ ]:
dfkat["structureId"].value_counts()

In [ ]:
dfkat["classification"].value_counts()

In [ ]:
dfkat["experimentalTechnique"].value_counts()

In [ ]:
dfkat["macromoleculeType"].value_counts()

In [ ]:
dfkat["crystallizationMethod"].value_counts()

In [ ]:
dfkat["pdbxDetails"].value_counts()

In [ ]:
dfkat.isnull().sum()

In [ ]:
def eksik_deger_tablosu(df): 
    eksik_deger = df.isnull().sum()
    eksik_deger_yuzde = 100 * df.isnull().sum()/len(df)
    eksik_deger_tablo = pd.concat([eksik_deger, eksik_deger_yuzde], axis=1)
    eksik_deger_tablo_son = eksik_deger_tablo.rename(
    columns = {0 : 'Eksik Değerler', 1 : '% Değeri'})
    return eksik_deger_tablo_son

In [ ]:
eksik_deger_tablosu(dfkat)

In [ ]:
dfkat.isin(["?", "-", ",", "*", "/"]).sum()

### 1.2) Sürekli Değişkenler

In [ ]:
dfnum = df.select_dtypes(include = ["int64", "float64"])
dfnum

In [ ]:
dfnum["phValue"].isnull().sum()

In [ ]:
dfnum.columns

In [ ]:
dfnum.describe().T

In [ ]:
dfnum.count()

In [ ]:
dfnum.isnull().sum()

In [ ]:
dfnum.isin(["?", "-", ",", "*", "/"]).sum()

In [ ]:
eksik_deger_tablosu(dfnum)

**resolution**, **phValue**, **densityMatthews** üçünden biri target değişken olarak kullanılabilir.

# -------------------------------------

* Yayinlanma yili, int olarak gelmişti. datetime olarak değiştirdik.

In [ ]:
df["publicationYear"] = df['publicationYear'].astype('datetime64[ns]')

In [ ]:
df.info()

## 2) Verinin Görselleştirilmesi (Data visualization)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import missingno as msno

In [ ]:
df1 = pd.read_csv("../input/protein-data-set/pdb_data_no_dups.csv")

* **Gorsellestirme yapilacagi zaman degiskenlerin siniflarinin 10'dan az olmamasi onemlidir.Cunku bu durumun aksi gorsellestirmeyi zorlastirmakta ve gerekli bilgiyi elde ederken engel teskil etmektedir. Ayni zamanda kirilmalari gostermek adina dusuk siniflarin olmasi da onemlidir.**

* **Bu sebepten oturu Classification degiskenin en cok sikliga sahip 7 sinifini alarak devam ediyoruz.**

In [ ]:
df2 =  df1[(df1['classification'] == "HYDROLASE") | (df1['classification'] == "TRANSFERASE")
           | (df1['classification'] == "OXIDOREDUCTASE") | (df1['classification'] == "LYASE") 
           | (df1['classification'] == "IMMUNE SYSTEM") | (df1['classification'] == "TRANSCRIPTION") | (df1['classification'] == "TRANSPORT PROTEIN")]

In [ ]:
df2["classification"].value_counts().plot.barh()

# Bagimli degisken olarak gordugumuz Classification degiskeni ile gorsellestirmeye basliyoruz

In [ ]:
sns.barplot( x = "classification", y = df2.classification.index, data = df2 )

In [ ]:
df2.groupby(["classification"])["structureMolecularWeight",].std()

# Bu group by isleminin amaci ise bir sonraki grafikte gorecegimiz barplot grafigindeki ince sapmalari belirten cubuklarin daha net anlasilmasi icindir.

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.barplot( x = "classification", y = "structureMolecularWeight", data = df2 )

# Classification degiskenine ait siniflarin Molekul agirligina gore sirlamasini gorebiliyoruz
# Yani en fazla molekul agirlina sahip olan sinif OXIDOREDUCTASE olarak gozukmekte
# Bu dagilimda en fazla standart sapmaya sahip sinif ise TRANSPORT PROTEIN olarak gozlenmekte 

* **Bir group by islemi yaparak barplot yapmadan once siniflarin molekul tipine gore agirliklarinin ortalamasini gorelim**
* **Boylece gorseldeki dagilimin da matematiksel kismini net bir sekilde gormus olacagiz**

In [ ]:
df2.groupby(["classification","macromoleculeType"])["structureMolecularWeight"].mean()

In [ ]:
plt.figure(figsize=(15,9))
ax = sns.barplot( x = "classification", y = "structureMolecularWeight",hue="macromoleculeType", data = df2 )

# Bir onceki grafikten farkli olarak bu grafikte ele olay ikinci bir kirilimi eklemis olmak
# Protein siniflari X eksinin de iken Y eksenin de proteinlerin molekuler agirliklari var
# Ve ayni zaman da bu siniflarin kendi icerisinde dagilimlari var ve  bu proteinlerin molekul tiplerine gore gozlenmekte
# Simdi Yukari daki GROUP BY islemine baktimiz da aslinda bu dagilimlarin saglamasi da gozukmekte 

In [ ]:
(sns
 .FacetGrid(df2,
            hue= "classification",
           height= 7.5,
           xlim= (0, 300000))
 .map(sns.kdeplot, "structureMolecularWeight",shade = True)
 .add_legend())

# Proteinlerin ait olduklari siniflara da molekul tipine gore frekanslarini gorebiliyoruz
# Molekul agirliginin siniflara gore frekanslarinin daha farkli oldugunu anlayabiliyoruz
# Ornegin sinif TRANSCRIPTION olan proteinlerin molekul agirliklari 0-5000 arasinda iken frekans yogunlugu tavan yapmis durumda
# Daha sonra 5000-10000 molekul agirliginda ise IMMUNE SYSTEM sinifina ait proteinler frekans sikligi olarak en fazla olan duruyor
# Bir yerden sonra molekul agirli artan bir trende burundugunde artik tum siniflarin her molekul agirliginda neredeyse ayni frekans
## yogunluguna sahip oldugunu gorebiliyoruz. 

In [ ]:
sns.pairplot(df2,kind="reg")

# Bu gorsel ile suna deginmis olduk Scatter Plot bilindigi uzere sacilim ve bize korelasyon hakkinda bilgi vermekte.
# Ayni zamanda degiskenlerin birbiri ile olan iliskisini de gostermek acisindan bir regresyon egrisi ciziyoruz. 
# Ortaya cikan grafikte cok fazla detay oldugu icin bu grafiksel sonuclarin birde istatistiksel kismini yani korelasyon kismini ele alip yorum yapalim.

In [ ]:
df2.corr()

# Olayin istatistiksel kismina gecildiginde iki sonuc goze carpmakta 0.349965 ve 0.990821
# Iste bu iki sonuc residueCount - resolution arasinda pozitife yakin bir iliski oldugunu gostermekte.
# Kalintilar ile cozulme arasinda artan bir iliski gorulmekte ama fazla denemez

# Daha sonra structureMolecularWeight - residueCount arasinda yuksek pozitif yonlu bir iiski oldugu gozlenmekte
# Yani molekul agirligi artarken cozulme degeri de artmakta.  

* **Iki adet sonucumuz vardi 0,99 ve 0,34 sirasiyla bunlari grafik uzerinde de inceleyelim** 

In [ ]:
plt.figure(figsize=(12,6))
ax = sns.scatterplot(x = "structureMolecularWeight", y = "residueCount" ,hue="classification",data=df2)

# Bu 0,99 iliskiye bakarken bir de fazladan bilgi almak adina HUE ile Classification yani siniflari da ekliyoruz.
# Molekul siniflarina gore molekul agirli ve kalinti(tortular) arasindaki iliskinin neye gore arttigi hakkinda biraz daha bilgi sahibi olmus olduk.

In [ ]:
plt.figure(figsize=(12,6))
ax = sns.scatterplot(x = "resolution", y = "residueCount" ,hue="classification",data=df2)

# Ikinci yuksek sayilabilecek 0,34 iliski incelendigin de ise bunlarin hangi siniflar bazli oldugu ve yayilimda yine hangilerinin etkili oldugunu anlayabiliriz.
# Genel olarak bir yere kadar iki degisken arasindaki iliski birlikte artarken ardindan bu trend durmus.
# Bunun arkasindaki detaylar ise biraz da is bilgisine dayanmaktadir. 


### 2.1) Missing Value Görselleştirme

In [ ]:
df.isnull().sum()

In [ ]:
msno.bar(df) 
#Grafikteki barların yüksekliği total veriye oranla orada olan sayıyı veriyor. Örneğin structerId de boş değer yok diyebiliriz.

In [ ]:
msno.heatmap(df) 

### 2.2) phValue Görselleştirmesi

In [ ]:
baz = df1[df1["phValue"]  > 7 ]["phValue"]

In [ ]:
notr = df1[df1["phValue"]  == 7 ]["phValue"]

In [ ]:
asit = df1[df1["phValue"]  < 7 ]["phValue"]

In [ ]:
sozluk = {"Asit": asit,
          "Notr": notr,
          "Baz": baz
         }

In [ ]:
phtablo = pd.DataFrame(sozluk)

In [ ]:
phtablo["Notr"].value_counts()

In [ ]:
phtablo.count()

In [ ]:
phtablo.count().plot.barh()

## 3) Aykırı Değer Temizleme(Outlier Cleaning)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

In [ ]:
data = pd.read_csv("../input/protein-data-set/pdb_data_no_dups.csv")
data.head(2)

In [ ]:
y = data[["classification"]]
y

In [ ]:
x = data.drop("classification", axis = 1)
x.head(2)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
columns = x_train.select_dtypes(["int64", "float64"])
columns

In [ ]:
del columns["phValue"]
del columns["publicationYear"]
columns

In [ ]:
columns.columns

In [ ]:
lower_and_upper = {}
x_train_copy = x_train.copy()
for col in columns.columns:
    q1 = x_train[col].describe()[4]
    q3 = x_train[col].describe()[6]
    iqr = q3 - q1
    lowerbound = q1 - (1.5*iqr)
    upperbound = q3 + (1.5*iqr)
    lower_and_upper[col] = (lowerbound, upperbound)
    x_train_copy.loc[(x_train_copy.loc[:,col] < lowerbound), col] = lowerbound * 0.75
    x_train_copy.loc[(x_train_copy.loc[:,col] > upperbound), col] = upperbound * 1.25

In [ ]:
lower_and_upper

In [ ]:
x_test_copy = x_test.copy()
for col in columns.columns:
    x_test_copy.loc[(x_test_copy.loc[:,col] < lower_and_upper[col][0]), col] = lower_and_upper[col][0] * 0.75
    x_test_copy.loc[(x_test_copy.loc[:,col] > lower_and_upper[col][1]), col] = lower_and_upper[col][1] * 1.25

In [ ]:
sns.boxplot(x_test_copy.residueCount)

In [ ]:
sns.boxplot(x_train_copy.residueCount)

### 3.1) Baskılama Yöntemi

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None

In [ ]:
df = pd.read_csv("../input/protein-data-set/pdb_data_no_dups.csv")

In [ ]:
df.info()

In [ ]:
df.describe().T

* **Outlier temizlenecegi zaman kategorik degiskenler disarida birakilarak numeric degiskenler uzerinden gidip ona gore Upper ve Lower degerler belirlenir.**

* **Bu nokta da bagimli degisken olarak bildiklerimiz kadariyla ele alacagimiz degisken DensityMatthews yani yogunluk olacaktir. Protenin molekul yogunlugu. Bu su demek degildir kesin olarak bagimli degisken DensityMatthews'dir diyemeyiz malum veri setinin is bilgisine pek hakim degiliz. Biz bildiklerimiz kadariyla bir baglanti kurup bunun uzerinden bir cikarim yapiyoruz.**

In [ ]:
dfnum = df.select_dtypes(include = ["int64", "float64"])
#dfnum = dfnum.dropna()

In [ ]:
y = dfnum.densityMatthews
X = dfnum.drop('densityMatthews',axis=1)

In [ ]:
y

In [ ]:
X

* **Bu kisimda Train icin veri setinin %80 degerini Test icin ise %20 degerini parcaladik.**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 123)

* **Train deki birkac degiskeni gorsellestirelim boxplot ile**

In [ ]:
sns.boxplot(x = "crystallizationTempK",data = X_train)


In [ ]:
sns.boxplot(x ="structureMolecularWeight",data = X_train)

In [ ]:
sns.boxplot(x ="residueCount",data = X_train)

* **Simdi sirada en sik kullanilan yontemlerden biri olan Quantiller a gore alt ve ust sinir belirleyecegiz**

In [ ]:
Q1 = X_train.quantile(0.25)
Q3 = X_train.quantile(0.75)
IQR = Q3 - Q1 

In [ ]:
Q1

In [ ]:
Q3

In [ ]:
IQR

In [ ]:
alt_sinir = Q1 - 1.5 * IQR
ust_sinir = Q3 + 1.5 * IQR


* **Belirledigimiz alt ve ust sinirlara gore outlier degerlerini bulup onlari farkli metodlar ile isleyecegiz.**

In [ ]:
alt_sinir

In [ ]:
ust_sinir

* **Toplu sekilde aykiri deger doldurma ve degisken degisken aykiri deger doldurma yontemleri vardir. Biz bu adimda degiskenleri tek tek ele alip aykiri degerleri dolduracagiz.**

In [ ]:
residueCount = X_train["residueCount"]

In [ ]:
(residueCount < alt_sinir.residueCount) | (residueCount > ust_sinir.residueCount)

In [ ]:
RCount_upper = (residueCount > ust_sinir.residueCount)

In [ ]:
RCount_lower = (residueCount < alt_sinir.residueCount)

* **Alt ust sinirlari belirledik simdi baskilama yontemi(alt sinir altini alt sinira esitle, ust sinirin ustunu de ust sinira esitle) ile bu aykiri degerleri dolduralim.** 

In [ ]:
# Ust siniri dolduruyoruz

X_train["residueCount"][RCount_upper] = ust_sinir.residueCount
X_train["residueCount"][RCount_upper]

# Yeni "residueCount" degiskenimiz de ust sinirdaki tum degerleri doldurduk 

In [ ]:
# Ayni sekilde ust siniri da baskilama yontemi ile dolduruyoruz

X_train["residueCount"][RCount_lower] = alt_sinir.residueCount
X_train["residueCount"][RCount_lower]

# Alt sinirin altinda deger olmadigi icin bos bir seri goruyoruz 

* **3 Adet degisken gorsellestirmistik oncesinde sirasiyla hepsinin outlier degerlerini temizleyip tekrardan ortaya cikan gorsellere bakalim.**

In [ ]:
sns.boxplot(x ="residueCount",data = X_train)

In [ ]:
# Bir diger degiskene geciyoruz "crystallizationTempK"

crystallizationKelvin = X_train["crystallizationTempK"]

In [ ]:
(crystallizationKelvin < alt_sinir.crystallizationTempK) | (crystallizationKelvin > ust_sinir.crystallizationTempK)

In [ ]:
upper = (crystallizationKelvin > ust_sinir.crystallizationTempK)
lower = (crystallizationKelvin < alt_sinir.crystallizationTempK)

In [ ]:
X_train["crystallizationTempK"][upper] = ust_sinir.crystallizationTempK
X_train["crystallizationTempK"][upper]

In [ ]:
X_train["crystallizationTempK"][lower] = alt_sinir.crystallizationTempK
X_train["crystallizationTempK"][lower]

In [ ]:
sns.boxplot(x = "crystallizationTempK",data = X_train)

*  **Son olarak da diger degiskene geciyoruz MolekulAgirligi**

In [ ]:
MolekularWeight = X_train["structureMolecularWeight"]
MolekularWeight

In [ ]:
(MolekularWeight < alt_sinir.structureMolecularWeight) | (MolekularWeight > ust_sinir.structureMolecularWeight)

In [ ]:
upper = (MolekularWeight > ust_sinir.structureMolecularWeight)
lower = (MolekularWeight < alt_sinir.structureMolecularWeight)

In [ ]:
X_train["structureMolecularWeight"][upper] = ust_sinir.structureMolecularWeight
X_train["structureMolecularWeight"][upper]

In [ ]:
X_train["structureMolecularWeight"][upper] = alt_sinir.structureMolecularWeight
X_train["structureMolecularWeight"][upper]

In [ ]:
sns.boxplot(x = "structureMolecularWeight",data = X_train)

* **Ayni Alt ve Ust sinirlari Y_train kisminda da birkac degisken icin kullaniyoruz**

In [ ]:
alt_sinir

In [ ]:
ust_sinir

In [ ]:
# Test kismindaki degsikenler icinde TRAIN de gecerli olan ALT ve UST sinir degerlerini kullanarak 3 adet degiskenin OUTLIER degerlerini dolduralim

X_test

In [ ]:
sns.boxplot(x = "crystallizationTempK",data = X_test)

In [ ]:
sns.boxplot(x = "structureMolecularWeight",data = X_test)

In [ ]:
sns.boxplot(x = "residueCount",data = X_test)

In [ ]:
crystallizationKelvin = X_test["crystallizationTempK"]

In [ ]:
(crystallizationKelvin < alt_sinir.crystallizationTempK) | (crystallizationKelvin > ust_sinir.crystallizationTempK)

In [ ]:
upper = (crystallizationKelvin > ust_sinir.crystallizationTempK)
lower = (crystallizationKelvin < alt_sinir.crystallizationTempK)

In [ ]:
X_test["crystallizationTempK"][upper] = ust_sinir.crystallizationTempK
X_test["crystallizationTempK"][upper]

In [ ]:
X_test["crystallizationTempK"][lower] = alt_sinir.crystallizationTempK
X_test["crystallizationTempK"][lower]

In [ ]:
sns.boxplot(x = "crystallizationTempK",data = X_test)


* **Ilk degisken bitti simdi sirayla diger Test setindeki 2 degiskene de Traindaki alt ve ust sinirlari uygulayarak devam ediyoruz.**

In [ ]:
MolekulAgirligi = X_test["structureMolecularWeight"]

In [ ]:
(MolekulAgirligi < alt_sinir.structureMolecularWeight) | (MolekulAgirligi > ust_sinir.structureMolecularWeight)

In [ ]:
upper = (MolekulAgirligi > ust_sinir.structureMolecularWeight)
lower = (MolekulAgirligi < alt_sinir.structureMolecularWeight)

In [ ]:
X_test["structureMolecularWeight"][upper] = ust_sinir.structureMolecularWeight
X_test["structureMolecularWeight"][upper]

In [ ]:
X_test["structureMolecularWeight"][upper] = alt_sinir.structureMolecularWeight
X_test["structureMolecularWeight"][upper]

In [ ]:
sns.boxplot(x = "structureMolecularWeight",data = X_test)

In [ ]:
residueCount = X_test["residueCount"]

In [ ]:
(residueCount < alt_sinir.residueCount) | (residueCount > ust_sinir.residueCount)

In [ ]:
upper = (residueCount > ust_sinir.residueCount)
lower = (residueCount < alt_sinir.residueCount)

In [ ]:
X_test["residueCount"][upper] = ust_sinir.residueCount
X_test["residueCount"][upper]

In [ ]:
X_test["residueCount"][lower] = alt_sinir.residueCount
X_test["residueCount"][lower]

In [ ]:
sns.boxplot(x = "residueCount",data = X_test)

## 4) Eksik Veri Doldurma

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
pd.options.mode.chained_assignment = None

In [ ]:
data = pd.read_csv("../input/protein-data-set/pdb_data_no_dups.csv")
data.head(2)

* Sayısal değişkenleri seçtik. phValue ve publicationYear kolonlarını kaldırdık.

In [ ]:
df = data.select_dtypes(["int64", "float64"])

In [ ]:
df.drop(columns=['publicationYear', 'phValue'], inplace = True)

* Bir satırda, 6 kolonda 3ten fazla bos deger var ise o satırı sildik. O verileri doldursakda bizim veri setimize uygun şekilde dolduramayız diye düşündüm

In [ ]:
df.isnull().sum()

In [ ]:
(df.isnull().sum(axis = 1) > 3).sum()

In [ ]:
df.loc[df.isnull().sum(axis = 1) > 3]

In [ ]:
df = df.loc[df.isnull().sum(axis = 1) <= 3]

In [ ]:
df

In [ ]:
df.isnull().sum()

### resolution

In [ ]:
df["resolution"].isnull().sum()

In [ ]:
df["resolution"].describe()

In [ ]:
df['resolution'].fillna(df.resolution.mean(), inplace = True)

In [ ]:
df["resolution"].isnull().sum()

In [ ]:
df["resolution"].describe()

### crystallizationTempK

In [ ]:
df["crystallizationTempK"].isnull().sum()

In [ ]:
df["crystallizationTempK"].describe()

In [ ]:
df['crystallizationTempK'].fillna(df.crystallizationTempK.mean(), inplace = True)

In [ ]:
df["crystallizationTempK"].isnull().sum()

In [ ]:
df["crystallizationTempK"].describe()

### densityMatthews

In [ ]:
df["densityMatthews"].isnull().sum()

In [ ]:
df["densityMatthews"].describe()

In [ ]:
df['densityMatthews'].fillna(df.densityMatthews.mean(), inplace = True)

In [ ]:
df["densityMatthews"].isnull().sum()

In [ ]:
df["densityMatthews"].describe()

### densityPercentSol

In [ ]:
df["densityPercentSol"].isnull().sum()

In [ ]:
df["densityPercentSol"].describe()

In [ ]:
df['densityPercentSol'].fillna(df.densityPercentSol.mean(), inplace = True)

In [ ]:
df["densityPercentSol"].isnull().sum()

In [ ]:
df["densityPercentSol"].describe()

* **Tüm Eksikler giderilmiş oldu**

In [ ]:
df.isnull().sum()